<a href="https://colab.research.google.com/github/Jcain17/Intro_to_ML/blob/main/ML_HW5_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
%matplotlib inline
import torch
import numpy as np
import torch.optim as optim
from google.colab import drive
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [37]:
drive.mount('/content/drive')

file_path = '/content/drive/My Drive/Housing.csv'
housing = pd.DataFrame(pd.read_csv(file_path))
housing.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [38]:
varlist2 = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']

def binmap(x):
   return x.map({'yes': 1, 'no': 0})

housing[varlist2] = housing[varlist2].apply(binmap)
housing.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,furnished
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,furnished
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,semi-furnished
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,furnished
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,furnished


In [39]:
sc_X = StandardScaler()
varlist1 = ['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'parking', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
housing[varlist1] = sc_X.fit_transform(housing[varlist1])

In [40]:
varlist = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']

X = housing[varlist].values
Y = housing['price']

In [41]:
X,Y

(array([[ 1.04672629,  1.40341936,  1.42181174, ..., -0.2192645 ,
          1.4726183 ,  1.80494113],
        [ 1.75700953,  1.40341936,  5.40580863, ..., -0.2192645 ,
          1.4726183 , -0.55403469],
        [ 2.21823241,  0.04727831,  1.42181174, ..., -0.2192645 ,
         -0.67906259,  1.80494113],
        ...,
        [-0.70592066, -1.30886273, -0.57018671, ..., -0.2192645 ,
         -0.67906259, -0.55403469],
        [-1.03338891,  0.04727831, -0.57018671, ..., -0.2192645 ,
         -0.67906259, -0.55403469],
        [-0.5998394 ,  0.04727831, -0.57018671, ..., -0.2192645 ,
         -0.67906259, -0.55403469]]),
 0      4.566365
 1      4.004484
 2      4.004484
 3      3.985755
 4      3.554979
          ...   
 540   -1.576868
 541   -1.605149
 542   -1.614327
 543   -1.614327
 544   -1.614327
 Name: price, Length: 545, dtype: float64)

In [42]:
x = torch.tensor(X)
y = torch.tensor(Y)
x,y

(tensor([[ 1.0467,  1.4034,  1.4218,  ..., -0.2193,  1.4726,  1.8049],
         [ 1.7570,  1.4034,  5.4058,  ..., -0.2193,  1.4726, -0.5540],
         [ 2.2182,  0.0473,  1.4218,  ..., -0.2193, -0.6791,  1.8049],
         ...,
         [-0.7059, -1.3089, -0.5702,  ..., -0.2193, -0.6791, -0.5540],
         [-1.0334,  0.0473, -0.5702,  ..., -0.2193, -0.6791, -0.5540],
         [-0.5998,  0.0473, -0.5702,  ..., -0.2193, -0.6791, -0.5540]],
        dtype=torch.float64),
 tensor([ 4.5664e+00,  4.0045e+00,  4.0045e+00,  3.9858e+00,  3.5550e+00,
          3.2553e+00,  2.8807e+00,  2.8807e+00,  2.7309e+00,  2.6934e+00,
          2.6934e+00,  2.6297e+00,  2.4312e+00,  2.3938e+00,  2.3938e+00,
          2.3188e+00,  2.3188e+00,  2.2439e+00,  2.2065e+00,  2.1877e+00,
          2.1315e+00,  2.0941e+00,  2.0754e+00,  2.0754e+00,  2.0379e+00,
          2.0192e+00,  1.9780e+00,  1.9443e+00,  1.9443e+00,  1.9443e+00,
          1.9443e+00,  1.9443e+00,  1.8881e+00,  1.8319e+00,  1.7944e+00,
          1

In [43]:
n_samples = x.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

In [44]:
train_x = x[train_indices]

val_x = x[val_indices]

train_xu = x[train_indices]

val_xu = x[val_indices]

train_y = y[train_indices]

val_y = y[val_indices]

In [45]:
def lin(x, w0, w1, w2, w3, w4, w5, w6, w7, w8, w9, w10):
  return w0 + x[:,0]*w1 + x[:,1]*w2 + x[:,2]*w3 + x[:,3]*w4 + x[:,4]*w5 + x[:,5]*w6 + x[:,6]*w7 + x[:,7]*w8 + x[:,8]*w9 + x[:,9]*w10

In [46]:
def loss_fun(tp, tc):
  diff = (tp - tc)**2
  return diff.mean()

In [47]:
w0 = torch.ones(())
w1 = torch.ones(())
w2 = torch.ones(())
w3 = torch.ones(())
w4 = torch.ones(())
w5 = torch.ones(())
w6 = torch.ones(())
w7 = torch.ones(())
w8 = torch.ones(())
w9 = torch.ones(())
w10 = torch.ones(())

tp = lin(train_x, w0, w1, w2, w3, w4, w5, w6, w7, w8, w9, w10)
tp

tensor([-1.9369e+00,  1.1378e+01,  3.4761e+00,  1.5842e+00, -4.8388e-01,
         9.1310e+00,  4.3752e+00,  5.0205e+00,  1.4922e-01,  5.0463e+00,
         5.1606e-01,  1.6488e+00,  3.9111e+00,  4.9302e+00, -3.8417e+00,
         2.6181e+00,  1.0244e+01,  1.6185e+00,  1.1958e+00, -1.4524e+00,
         7.3596e-01,  3.7976e+00, -3.0616e+00, -6.9274e-01,  2.5666e+00,
        -2.1693e-01,  6.8328e+00, -5.0127e+00,  2.3432e-01,  8.7071e+00,
         4.4138e-01,  3.0755e-01, -2.5535e+00, -2.0292e+00,  7.8926e+00,
         5.1804e+00, -4.8005e+00, -1.1657e+00,  8.6881e+00, -6.2485e+00,
         7.3246e+00,  7.0308e+00, -8.1696e+00, -1.1547e+00, -1.1473e+00,
         1.5237e+01, -2.9204e+00,  1.1562e+01,  8.2098e+00, -1.0735e+00,
        -7.1739e+00,  3.6883e+00,  2.3998e+00, -1.7467e-01, -2.1430e+00,
         5.0585e+00,  4.1231e+00,  4.8559e-01, -1.8503e+00,  7.9466e+00,
         7.7194e+00,  3.6710e+00,  9.2527e-01, -4.8519e-01, -2.5040e+00,
        -6.3576e+00, -5.2156e+00, -5.0034e+00, -1.3

In [48]:
def training_loop(n_epochs, optimizer, params, train_t_u, val_t_u,
                  train_t_c, val_t_c):
    for epoch in range(1, n_epochs + 1):
        train_t_p = lin(train_t_u, *params)
        train_loss = loss_fun(train_t_p, train_t_c)

        val_t_p = lin(val_t_u, *params)
        val_loss = loss_fun(val_t_p, val_t_c)

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        if epoch <= 3 or epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
                  f" Validation loss {val_loss.item():.4f}")

    return params

In [49]:
params = torch.tensor([1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], requires_grad=True)
learning_rate = 0.001
optimizer = optim.SGD([params], lr=learning_rate)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    params = params,
    train_t_u = train_xu,
    val_t_u = val_xu,
    train_t_c = train_y,
    val_t_c = val_y)

Epoch 1, Training loss 1.9099, Validation loss 2.3604
Epoch 2, Training loss 1.9008, Validation loss 2.3500
Epoch 3, Training loss 1.8918, Validation loss 2.3397
Epoch 500, Training loss 0.4708, Validation loss 0.6646
Epoch 1000, Training loss 0.3439, Validation loss 0.4903
Epoch 1500, Training loss 0.3268, Validation loss 0.4585
Epoch 2000, Training loss 0.3242, Validation loss 0.4505
Epoch 2500, Training loss 0.3238, Validation loss 0.4478
Epoch 3000, Training loss 0.3237, Validation loss 0.4468
Epoch 3500, Training loss 0.3237, Validation loss 0.4464
Epoch 4000, Training loss 0.3237, Validation loss 0.4462
Epoch 4500, Training loss 0.3236, Validation loss 0.4461
Epoch 5000, Training loss 0.3236, Validation loss 0.4460


tensor([0.0099, 0.3000, 0.0536, 0.2627, 0.2019, 0.1334, 0.1263, 0.0624, 0.1298,
        0.0745, 0.2040], requires_grad=True)

In [50]:
params = torch.tensor([1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], requires_grad=True)
learning_rate = 0.001
optimizer = optim.Adam([params], lr=learning_rate)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    params = params,
    train_t_u = train_xu,
    val_t_u = val_xu,
    train_t_c = train_y,
    val_t_c = val_y)

Epoch 1, Training loss 1.9099, Validation loss 2.3604
Epoch 2, Training loss 1.9013, Validation loss 2.3499
Epoch 3, Training loss 1.8927, Validation loss 2.3394
Epoch 500, Training loss 0.6309, Validation loss 0.8406
Epoch 1000, Training loss 0.3871, Validation loss 0.5433
Epoch 1500, Training loss 0.3311, Validation loss 0.4635
Epoch 2000, Training loss 0.3240, Validation loss 0.4486
Epoch 2500, Training loss 0.3237, Validation loss 0.4463
Epoch 3000, Training loss 0.3236, Validation loss 0.4460
Epoch 3500, Training loss 0.3236, Validation loss 0.4459
Epoch 4000, Training loss 0.3236, Validation loss 0.4459
Epoch 4500, Training loss 0.3236, Validation loss 0.4459
Epoch 5000, Training loss 0.3236, Validation loss 0.4459


tensor([0.0099, 0.3003, 0.0530, 0.2627, 0.2027, 0.1334, 0.1260, 0.0621, 0.1304,
        0.0745, 0.2038], requires_grad=True)